In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 提示設計 - 最佳實務

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/language/prompts/intro_prompt_design.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory 標誌"><br> 在 Colab 中執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/language/prompts/intro_prompt_design.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub 標誌"><br> 在 GitHub 上檢視
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/language/prompts/intro_prompt_design.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI 標誌"><br> 在 Vertex AI 工作台中開啟
    </a>
  </td>
</table>


| | |
|-|-|
|作者 | [Polong Lin](https://github.com/polong-lin) |


## 概觀

此筆記本涵蓋提示工程的基本要素，包含一些最佳實務。

在 [官方文件](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview)中深入了解提示設計。


### 目標

在這個筆記本中，你會學習提示工程的最佳實務，也就是如何設計提示以提升回應品質。

這個筆記本涵蓋提示工程的最佳實務，如下：

- 要簡潔
- 要具體且定義明確
- 一次只詢問一項任務
- 將生成式任務轉換為分類任務
- 加入範例來提升回應品質


### 費用
本教學指南使用 Google
Cloud 的計費元件：

* Vertex AI Generative AI Studio

了解 [Vertex AI 定價](https://cloud.google.com/vertex-ai/pricing)
，並使用 [定價計算器](https://cloud.google.com/products/calculator/)
，根據預計用量產生成本估算。


### 安裝 Vertex AI SDK


In [ ]:
!pip install google-cloud-aiplatform protobuf==3.19.3 --upgrade --user

In [ ]:
!pip install -U google-cloud-aiplatform "shapely<2"

**僅限 Colab：** 執行以下單元格重新啟動核心或使用按鈕重新啟動核心。對於**Vertex AI Workbench** ，你可以使用頂端的按鈕重新啟動終端機。


In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### 驗證你的筆記本環境

- 如果你使用 **Colab** 來執行這個筆記本，請執行以下單元格並繼續。
- 如果你使用的是 **Vertex AI Workbench** ，在此處查看設定說明 [link](https://github.com/doggy8088/generative-ai/tree/main/setup-env).


In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

- 如果你是在本機開發環境中執行這個筆記本：
  - 安裝 [Google Cloud SDK](https://cloud.google.com/sdk)。
  - 取得驗證憑證。透過執行以下指令並遵循 oauth2 流程 (在此處進一步了解此指令 [here](https://cloud.google.com/sdk/gcloud/reference/beta/auth/application-default/login))，來建立本地憑證：

    ```bash
    gcloud auth application-default login
    ```


### 匯入函式庫


**僅限 Colab：** 執行下列Cell以初始化 Vertex AI SDK。對於 Vertex AI Workbench，不需要執行此操作。


In [ ]:
import vertexai

PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

vertexai.init(project=PROJECT_ID, location=REGION)

In [ ]:
from vertexai.language_models import TextGenerationModel
from vertexai.language_models import ChatModel

### 載入模型


In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

## Prompt 工程最佳實務


提示工程的重點在於如何設計提示，讓回應符合你確實希望看到的內容。

使用「不花俏」提示的想法是將提示中的雜訊降到最低，以減少 LLM 誤解提示意圖的可能性。以下是一些如何設計「不花俏」提示的指南。

在此部分，你將涵蓋在設計提示時的下列最佳實務做法：

* 簡潔扼要
* 具體且定義明確
* 一次詢問一項任務
* 加入範例以提升回應品質
* 將生成式任務轉換為分類任務以提升安全性


### 簡潔


🛑 不建議。以下提示過於冗長。


In [ ]:
prompt = "What do you think could be a good name for a flower shop that specializes in selling bouquets of dried flowers more than fresh flowers? Thank you!"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

✅ 推薦。以下提示直截了當、簡潔明了。


In [ ]:
prompt = "Suggest a name for a flower shop that sells bouquets of dried flowers"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

### 具體明確，定義明確


假設你想集思廣益描述地球的創意方法。


🛑 不建議。以下提示太過於一般性。


In [ ]:
prompt = "Tell me about Earth"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

✅ 推薦。以下提示具體且有明確定義。


In [ ]:
prompt = "Generate a list of ways that makes Earth unique compared to other planets"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

### 一次問一個任務


🛑 不建議。以下提示符包含兩個部分的問題，可以單獨提出的問題。


In [ ]:
prompt = "What's the best method of boiling water and why is the sky blue?"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

✅ 建議使用。以下提示一次詢問一個任務。


In [ ]:
prompt = "What's the best method of boiling water?"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

In [ ]:
prompt = "Why is the sky blue?"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

### 小心出現幻覺


儘管 LLM 已根據大量資料進行訓練，但它們會產生包含未根據真實或現實的陳述的文字；由於其記憶能力有限，因此 LLM 的這些回應通常被稱為「幻覺」。請注意，要求 LLM 提供引用並不能解決這個問題，因為有 LLM 提供錯誤或不準確引用的情況。處理幻覺是 LLM 面臨的一項基本挑戰，也是一個持續的研究領域，因此，必須了解 LLM 可能看似提供肯定、正確的陳述，但事實上卻是不正確的。

請注意，如果你打算將 LLM 用于創意用途，那麼幻覺實際上可能非常有用。


重複嘗試類似的提示。你可能會注意到有時它會自信但又不準確地說「第一個登陸月球的大象是 Luna」。


In [ ]:
prompt = "Who was the first elephant to visit the moon?"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

顯然聊天機器人出現了幻覺，因為從來沒有大象飛到月球上。但我們如何防止這些不適當問題的出現，更具體地說，如何減少幻覺？

有一種可能的方法稱為確定適當回應 (DARE) 提示，它巧妙地使用 LLM 本身根據其任務來決定是否回答問題。

讓我們透過建立一個旅遊網站的聊天機器人並略加調整，看看它是如何運作的。


In [ ]:
chat_model = ChatModel.from_pretrained("chat-bison@002")

chat = chat_model.start_chat()
dare_prompt = """Remember that before you answer a question, you must check to see if it complies with your mission.
If not, you can say, Sorry I can't answer that question."""

print(
    chat.send_message(
        f"""
Hello! You are an AI chatbot for a travel web site.
Your mission is to provide helpful queries for travelers.

{dare_prompt}
"""
    )
)

假設我們請教關於一個最有名的義大利景點之類的簡單問題。


In [ ]:
prompt = "What is the best place for sightseeing in Milan, Italy?"
print(chat.send_message(prompt))

現在讓我們假裝成一個不太友善的使用者，並詢問聊天機器人一個與旅遊無關的問題。


In [ ]:
prompt = "Who was the first elephant to visit the moon?"
print(chat.send_message(prompt))

你可以看到，DARE 指令新增了一層防護欄，以防聊天機器人偏離軌道。


### 將生成式任務變成分類任務以降低輸出變異性


#### 生成式任務將導致更高的輸出變異性


下列提示產生開放式的回應，適合用於腦力激盪，但回應內容變動率高。


In [ ]:
prompt = "I'm a high school student. Recommend me a programming activity to improve my skills."

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

#### 分類任務節省輸出變異性


以下提示會導致選擇，如果你想要更容易控制輸出，這可能會很有用。


In [ ]:
prompt = """I'm a high school student. Which of these activities do you suggest and why:
a) learn Python
b) learn Javascript
c) learn Fortran
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

### 透過加入範例來改善回應品質


改善回應品質的另一種方法是在你的提示中加入範例。LLM 會根據回應範例從背景中學習。通常，一個到五個範例 (拍攝) 就足以改善回應品質。加入太多範例可能會導致模型過度符合資料，並降低回應品質。

與傳統模型訓練類似，範例的品質與分配非常重要。挑選代表你希望模型學習的場景範例，並使範例的分配 (例如在分類中的每個類別的範例數) 與你的實際分配保持一致。


#### 零次提示


以下是一個零範例提示的範例，你在提示中沒有提供任何範例給 LLM。


In [ ]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment:
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

#### One-shot 提示


以下是一個單次提示範例，你在提示中提供一個範例給 LLM，作為你想要哪種類型回應的指導。


In [ ]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment:
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

#### Few-shot 提示


以下是一個少樣本提示範例，讓你在提示中為 LLM 提供一些範例，以提供你要什麼類型回應的指導。


In [ ]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment: negative

Tweet: Something surprised me about this video - it was actually original. It was not the same old recycled stuff that I always see. Watch it - you will not regret it.
Sentiment:
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

#### 選擇零次、一次、少次提示法


不同的提示技術取決於你的目標。零鏡頭提示較為開放且能提供具創意的答案，而一次鏡頭和少量鏡頭提示則可教授模型該如何表現，好讓你獲得更多可預測且與所提供範例一致的答案。
